# Load Preprocessed Envelopes
This notebook processes preprocessed data to generate attended and unattended envelopes.

In [4]:
import sys
# Add parent directory to path to import file_paths module
sys.path.insert(0, '..')
import file_paths
import helper_functions

import eelbrain
from scipy.io import loadmat

In [5]:
SUBJECTS = helper_functions.get_subjects()

for SUBJECT in SUBJECTS:
    
    # Make sure the destination directory exists
    dst_dir = file_paths.ENVELOPES_DIR / SUBJECT
    dst_dir.mkdir(exist_ok=True, parents=True)

    # Define destination file paths
    attended_dst = dst_dir / f'{SUBJECT}_attended_envelope.pickle'
    unattended_dst = dst_dir / f'{SUBJECT}_unattended_envelope.pickle'
    padded_attended_dst = dst_dir / f'{SUBJECT}_padded_attended_envelope.pickle'
    padded_unattended_dst = dst_dir / f'{SUBJECT}_padded_unattended_envelope.pickle'

    # If all files already exist, skip processing
    if attended_dst.exists() and unattended_dst.exists() and padded_attended_dst.exists() and padded_unattended_dst.exists():
        print(f"Files for {SUBJECT} already exist, skipping.")
        continue

    print("Saving ~", SUBJECT, "...")

    subject_data_file = helper_functions.get_subject_data_file(SUBJECT)
    mat = loadmat(subject_data_file, squeeze_me=True, struct_as_record=False)
    data = mat['data']

    # --- collect all trials ---
    att_envelope_trials = []
    unatt_envelope_trials = []
    padded_att_envelope_trials = []
    padded_unatt_envelope_trials = []

    for att_envelope, unatt_envelope in zip(data.wavA, data.wavB):  # 60 trials
        # Create time axis
        time_axis = eelbrain.UTS(0, 1 / data.fsample.wavA, len(att_envelope))

        # Save the attended envelope as an NDVar
        att_ndvar = eelbrain.NDVar(att_envelope, dims=(time_axis,), name='attended')
        # Save the unattended envelope as an NDVar
        unatt_ndvar = eelbrain.NDVar(unatt_envelope, dims=(time_axis,), name='unattended')

        # Create padded versions
        padded_att_ndvar = eelbrain.pad(att_ndvar, tstart=-0.100, tstop=att_ndvar.time[-1] + 1, name='padded_attended')
        padded_unatt_ndvar = eelbrain.pad(unatt_ndvar, tstart=-0.100, tstop=unatt_ndvar.time[-1] + 1, name='padded_unattended')

        att_envelope_trials.append(att_ndvar)
        unatt_envelope_trials.append(unatt_ndvar)
        padded_att_envelope_trials.append(padded_att_ndvar)
        padded_unatt_envelope_trials.append(padded_unatt_ndvar)

    # --- concatenate trials ---
    att_envelope_concat = eelbrain.concatenate(att_envelope_trials)
    unatt_envelope_concat = eelbrain.concatenate(unatt_envelope_trials)
    padded_att_envelope_concat = eelbrain.concatenate(padded_att_envelope_trials)
    padded_unatt_envelope_concat = eelbrain.concatenate(padded_unatt_envelope_trials)

    # Save all 4 versions
    eelbrain.save.pickle(att_envelope_concat, attended_dst)
    eelbrain.save.pickle(unatt_envelope_concat, unattended_dst)
    eelbrain.save.pickle(padded_att_envelope_concat, padded_attended_dst)
    eelbrain.save.pickle(padded_unatt_envelope_concat, padded_unattended_dst)
    print(f"Saved attended, unattended, padded attended, and padded unattended envelopes for {SUBJECT}.")


Files for S1 already exist, skipping.
Files for S2 already exist, skipping.
Files for S3 already exist, skipping.
Files for S4 already exist, skipping.
Files for S5 already exist, skipping.
Files for S6 already exist, skipping.
Files for S7 already exist, skipping.
Files for S8 already exist, skipping.
Files for S9 already exist, skipping.
Files for S10 already exist, skipping.
Files for S11 already exist, skipping.
Files for S12 already exist, skipping.
Files for S13 already exist, skipping.
Files for S14 already exist, skipping.
Files for S15 already exist, skipping.
Files for S16 already exist, skipping.
Files for S17 already exist, skipping.
Files for S18 already exist, skipping.


In [6]:
# Sanity check: load one of the saved files and print its shape
att_ndvar = eelbrain.load.unpickle(file_paths.ENVELOPES_DIR / "S1" / "S1_attended_envelope.pickle")
unatt_ndvar = eelbrain.load.unpickle(file_paths.ENVELOPES_DIR / "S1" / "S1_unattended_envelope.pickle")
padded_att_ndvar = eelbrain.load.unpickle(file_paths.ENVELOPES_DIR / "S1" / "S1_padded_attended_envelope.pickle")
padded_unatt_ndvar = eelbrain.load.unpickle(file_paths.ENVELOPES_DIR / "S1" / "S1_padded_unattended_envelope.pickle")
print(att_ndvar, unatt_ndvar, padded_att_ndvar, padded_unatt_ndvar)

<NDVar 'attended': 192000 time> <NDVar 'unattended': 192000 time> <NDVar 'padded_attended': 196200 time> <NDVar 'padded_unattended': 196200 time>
